# Авиакомпания Россия

## Импорт данных и алгоритмы

Импорт библиотек

In [1]:
import pandas as pd
import numpy as np

In [2]:
from russiya_airlines_solver import Solver
from russiya_airlines_interface import data_setting

Импорт исходных данных, который мы заслужили. При запуске ячейки должно открыться окно `tkinter` с интуитивным интерфейсом для импорта исходных данных. Данные о связках передаются в формате `XLS`. Данные о книгах передаются в формате `JSON`. Это **ВАЖНО!!!**

In [3]:
data, crew = data_setting()
print('Данные о книгах:\n', crew)
print('\nДанные о связках:\n')
data.head(5)

Данные о книгах:
           1       2       3       4       5       6
ВВЛ  125.54  122.84  129.97  127.26  123.32  130.64
МВЛ  119.88  117.30  124.10  121.51  117.76  124.75
СНГ  122.03  119.41  126.33  123.69  119.87  126.99

Данные о связках:



,День месяца,Время вылета,Связка,Назначение,Тип судна,Тип связи,Налет,Экипаж
0,1,02:55,route 1,city 61,А-320,ВВЛ,10:50,4
1,1,03:55,route 2,city 60,А-320,МВЛ,06:00,4
2,1,04:10,route 3,city 23,А-319,ВВЛ,03:00,3
3,1,04:40,route 4,city 32,А-319,МВЛ,08:35,3
4,1,05:00,route 5,city 8,А-319,ВВЛ,06:15,3


При запуске последовательно откроются два окна `tkinter`. В первом необходимо выбрать алгоритм и сортировку (рекомендуется выбирать A2, S1). Во втором задать веса критериев. По умолчанию все веса равны. Веса автоматически нормируются. 

In [4]:
sol = Solver(data, crew) 
print(sol.solve())

Время работы функции:	0.4475 сек

Решение записано в Solver.solutions;
Решение приведённое к формату Expert.xls записано в Solver.formated_solutions;
Спецификации решения сохранены в Solver.specifications;
Идентификатор решения: 1


Проверка распределения

In [5]:
identificator = 1
print('Значение целевой функции: {:.2f}'.format(Solver.solutions[identificator][0]))
for book in range(len(Solver.solutions[identificator][1])):
    print('Число связок на {}-ом рабочем столе: {}'.format(book, len(Solver.solutions[identificator][1][book])))

Значение целевой функции: 2.95
Число связок на 0-ом рабочем столе: 352
Число связок на 1-ом рабочем столе: 348
Число связок на 2-ом рабочем столе: 362
Число связок на 3-ом рабочем столе: 358
Число связок на 4-ом рабочем столе: 349
Число связок на 5-ом рабочем столе: 366


Экспорт решения

In [ ]:
Solver.formated_solutions[identificator].to_excel('solution.A2.S1.xls')

## Формирование отчёта

Импорт библиотек

In [4]:
from russiya_airlines_report import *
from russiya_airlines_interface import solution_import

### Пример с найденным решением

Анализ решения

In [5]:
solution = solution_import()
solution = solution[solution.columns[1:3]].join(solution[solution.columns[4:]] + 1).fillna(value=0)
solution.head()

,Связка,Вылет,1,2,3,4,5,6,7,8,...,22,23,24,25,26,27,28,29,30,31
1,route 1,02:55,0,0,0,6,0,4,0,0,...,0,0,0,3,0,6,0,0,0,0
2,route 1,02:55,5,2,5,0,1,0,0,6,...,4,4,2,0,2,0,0,3,1,4
3,route 71,03:10,0,0,0,0,5,0,0,0,...,0,0,0,0,2,0,0,0,0,0
4,route 71,03:10,0,0,6,4,0,6,6,0,...,0,0,3,1,0,2,3,0,0,4
5,route 2,03:55,5,1,1,2,6,5,6,6,...,5,2,1,3,4,2,1,1,4,5


In [6]:
t1 = table_round_trip(solution, data)
t1.head()

,ВВЛ: БП,ВВЛ: Связки,МВЛ: БП,МВЛ: СВ,СНГ: БП,СНГ: Связки,Всего связок,Всего носных связок
0,798.0,247.0,305.0,87.0,77.0,22.0,356.0,62.0
1,781.0,241.0,298.0,85.0,74.0,21.0,347.0,61.0
2,814.0,251.0,317.0,90.0,80.0,23.0,364.0,66.0
3,805.0,247.0,308.0,88.0,76.0,22.0,357.0,64.0
4,778.0,241.0,297.0,85.0,76.0,22.0,348.0,61.0


In [7]:
t2 = table_flight_time(solution, data)
t2.head()

,ВВЛ: Планируемый налёт,ВВЛ: Планируемый ночной налёт,МВЛ: Планируемый налёт,МВЛ: Планируемый ночной налёт,СНГ: Планируемый налёт,СНГ: Планируемый ночной налёт
1,32.8905,10.751,16.7709,4.14734,5.19528,1.79899
2,33.126,10.5558,16.8553,4.13313,5.21028,1.85183
3,32.0154,10.613,16.8196,4.17404,5.35888,1.86988
4,32.6851,10.2412,16.7186,4.12351,5.12686,1.77514
5,33.07,11.036,16.6411,4.04693,5.3314,1.85028


In [8]:
t3 = table_flight_on_direction(solution, data)
t3.head()

,1,2,3,4,5,6,All,delta
city 41,6.0,5.0,5.0,5.0,5.0,5.0,31.0,1.0
city 2,11.0,10.0,10.0,11.0,10.0,10.0,62.0,1.0
city 9,16.0,15.0,16.0,15.0,15.0,16.0,93.0,1.0
city 1,6.0,5.0,5.0,5.0,5.0,5.0,31.0,1.0
city 33,5.0,5.0,6.0,5.0,5.0,5.0,31.0,1.0


In [9]:
t4 = table_flight_on_date(solution, data)
t4.head()

,1,2,3,4,5,6,All,delta
1,11.0,11.0,11.0,11.0,11.0,11.0,66.0,0.0
2,11.0,11.0,12.0,11.0,11.0,12.0,68.0,1.0
3,12.0,11.0,12.0,11.0,11.0,12.0,69.0,1.0
4,11.0,11.0,12.0,11.0,11.0,11.0,67.0,1.0
5,12.0,11.0,13.0,12.0,11.0,11.0,70.0,2.0


In [10]:
t5 = table_flight_on_plane(solution, data)
t5

,1,2,3,4,5,6,All,delta
А-319,244.0,236.0,247.0,241.0,242.0,244.0,1454.0,11.0
А-320,112.0,111.0,117.0,116.0,106.0,119.0,681.0,13.0


### Пример с решением эксперта

Анализ решения

In [7]:
solution = solution_import().fillna(value=0)
solution.columns = list(map(str, solution.columns))
solution.head()

,Связка,Вылет,Тип,1,2,3,4,5,6,7,...,22,23,24,25,26,27,28,29,30,31
1,route 8,05:10,А 320,2,2,5,2,2,3,1,...,6,4,2,6,6,5,2,5,5,3
2,route 28,11:20,А 319,6,4,6,1,5,5,3,...,6,4,1,2,3,4,5,5,5,2
3,route 66,22:45,А 319,5,3,1,4,3,2,1,...,4,3,5,6,4,1,2,3,4,6
4,route 31,11:55,А 319,0,0,4,3,4,1,5,...,0,0,4,5,1,2,4,0,0,1
5,route 31,12:10,А 319,1,5,0,0,0,0,0,...,1,1,0,0,0,0,0,4,4,0


In [8]:
t1 = table_round_trip(solution, data)
t1.head()

AttributeError: 'Series' object has no attribute 'split'

In [15]:
t2 = table_flight_time(solution, data)
t2.head()

,ВВЛ: Планируемый налёт,ВВЛ: Планируемый ночной налёт,МВЛ: Планируемый налёт,МВЛ: Планируемый ночной налёт,СНГ: Планируемый налёт,СНГ: Планируемый ночной налёт
1,1283:00,382:02,574:15,42:12,197:50,92:31
2,1218:25,391:58,574:30,45:36,172:40,88:49
3,1241:35,358:20,546:25,38:13,177:40,89:35
4,1256:40,421:39,595:25,49:56,177:35,82:32
5,1224:40,380:23,586:10,42:00,177:20,90:00


In [16]:
t3 = table_flight_on_direction(solution, data)
t3.head()

,1,2,3,4,5,6,All,delta
city 2,8.0,9.0,12.0,12.0,10.0,11.0,62.0,4.0
city 14,15.0,13.0,14.0,14.0,13.0,14.0,83.0,2.0
city 21,4.0,8.0,0.0,6.0,5.0,8.0,31.0,8.0
city 5,4.0,6.0,8.0,4.0,4.0,5.0,31.0,4.0
city 42,0.0,2.0,3.0,0.0,3.0,2.0,10.0,3.0


In [17]:
t4 = table_flight_on_date(solution, data)
t4.head()

,1,2,3,4,5,6,All,delta
1,13.0,11.0,10.0,11.0,9.0,12.0,66.0,4.0
2,11.0,9.0,14.0,12.0,11.0,11.0,68.0,5.0
3,9.0,16.0,11.0,8.0,13.0,12.0,69.0,8.0
4,13.0,10.0,12.0,12.0,8.0,12.0,67.0,5.0
5,10.0,13.0,12.0,13.0,12.0,10.0,70.0,3.0


In [18]:
t5 = table_flight_on_plane(solution, data)
t5

,1,2,3,4,5,6,All,delta
А-319,236.0,239.0,254.0,245.0,237.0,243.0,1454.0,18.0
А-320,122.0,116.0,102.0,109.0,116.0,116.0,681.0,20.0


Конец файла